In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
violations_list = pd.read_csv("Codes' List of Violations 15 aug 2017-2.csv")

In [7]:
violations_list

,VIOLATION_ID,ORG_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT,TABLE_NAME,EXPIRED_FLAG,DATE_EXPIRED,CREATED_BY,DATE_CREATED,MODIFIED_BY,DATE_MODIFIED,FEE_SETUP_ID,Unnamed: 14
0,1,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,NaN,N,NaN,1.0,03:52.6,NaN,NaN,NaN,NaN
1,2,1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER,NaN,N,NaN,1.0,03:52.6,NaN,NaN,NaN,NaN
2,4,1,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT,NaN,N,NaN,1.0,03:52.6,NaN,NaN,NaN,NaN
3,5,1,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT,NaN,N,NaN,1.0,03:52.6,NaN,NaN,NaN,NaN
4,6,1,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE,NaN,N,NaN,1.0,03:52.6,NaN,NaN,NaN,NaN
5,7,1,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU,NaN,N,NaN,1.0,03:52.6,NaN,NaN,NaN,NaN
6,8,1,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG,NaN,N,NaN,1.0,03:52.6,NaN,NaN,NaN,NaN
7,9,1,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT,NaN,N,NaN,1.0,03:52.6,NaN,NaN,NaN,NaN
8,10,1,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP,NaN,N,NaN,1.0,03:52.6,NaN,NaN,NaN,NaN
9,11,1,CERTOCC,Certificate of Occupancy Required,Section 16.24.130 – Certificate of Occupancy R...,CAAH_CERT_OCC,NaN,N,NaN,1.0,03:52.6,NaN,NaN,NaN,NaN
